The first three blocks set up the library, kernels&functions, and the data matrices
Then choose kernel to benchmark

In [2]:
using Random, Distributions
using CUDA
using BenchmarkTools, Base.Threads
using PyPlot

In [3]:
#Initialize Kernels

#Method 1 of Vd
#A += (1-ϕ)* Vd0
function sumdef1(sumdef,Vd,Vd0,V0,ϕ,β,P)
    #sumdef = CUDA.zeros(Ny)
    A = ϕ* V0[:,1]
    A += (1-ϕ)* Vd0
    A.= ϕ.* V0[:,1] .+ (1-ϕ).* Vd0
    temp = P
    temp .*= CUDA.transpose(A)
    temp .*= β
    #temp = β* P .* CUDA.transpose(A)
    sumdef += reduce(+, temp, dims=2) #This gives Vd
    #Then do a value transport to Vd
    Vd = sumdef
end

#line 7.1 Intitializing U((1-τ)iy) to each Vd[iy]
function def_init(sumdef,τ,Y,α)
    iy = threadIdx().x
    stride = blockDim().x
    for i = iy:stride:length(sumdef)
        sumdef[i] = CUDA.pow(exp((1-τ)*Y[i]),(1-α))/(1-α)
    end
    return
end

#adding expected value to sumdef
function def_add(matrix, P, β, V0, Vd0, ϕ, Ny)
    y = (blockIdx().x-1)*blockDim().x + threadIdx().x
    iy = (blockIdx().y-1)*blockDim().y + threadIdx().y

    if (iy <= Ny && y <= Ny)
        matrix[iy,y] = β* P[iy,y]* (ϕ* V0[y,1] + (1-ϕ)* Vd0[y])
        #Note memory transfer of matrices of P and Vd0 are not optimal
    end
    return
end

#Method 2 of Vd
function sumdef2(sumdef) #Calculate sumdef in a kernel
    @cuda threads=threadcount blocks=blockcount def_init(sumdef,τ,Y,α)
    temp = CUDA.zeros(Ny,Ny)
    blockcount = (ceil(Int,Ny/10),ceil(Int,Ny/10))
    @cuda threads=threadcount blocks=blockcount def_add(temp, P, β, V0, Vd0, ϕ, Ny)
    sumdef += reduce(+, temp, dims=2)
end

#@benchmark sumdef1(sumdef) #240.6 μs
#@benchmark sumdef2(sumdef)
#----

#Calculate Cost Matrix C
function vr_C(Ny,Nb,Y,B,Price0,P,C)
    ib = (blockIdx().x-1)*blockDim().x + threadIdx().x
    iy = (blockIdx().y-1)*blockDim().y + threadIdx().y

    if (ib <= Nb && iy <= Ny)
        for b in 1:Nb
            C[iy,ib,b] = -Price0[iy,b]*B[b] + CUDA.exp(Y[iy]) + B[ib]
        end
    end
end

#map C -> U(C), then add β*sumret
function vr_C2(Ny,Nb,Vr,V0,Y,B,Price0,P,C,C2,sumret,α)
    ib = (blockIdx().x-1)*blockDim().x + threadIdx().x
    iy = (blockIdx().y-1)*blockDim().y + threadIdx().y

    if (ib <= Nb && iy <= Ny)
        for b in 1:Nb
            if C[iy,ib,b] > 0
                c = C[iy,ib,b]
                C2[iy,ib,b] = CUDA.pow(c,(1-α)) / (1-α) + B[ib] - Price0[iy,b]*B[b] #Note CUDA.pow only support certain types, need to cast constant to Float32 instead of Float64
            end
        end
    end
end

#----
#Calcuate sumret[iy,ib,b]
function vr_sumret(Ny,Nb,V0,P,sumret)
    ib = (blockIdx().x-1)*blockDim().x + threadIdx().x
    iy = (blockIdx().y-1)*blockDim().y + threadIdx().y

    if (ib <= Nb && iy <= Ny)
        for b in 1:Nb
            sumret[iy,ib,b] = 0
            for y in 1:Ny
                sumret[iy,ib,b] += P[iy,b]*V0[y,b]
            end
        end
    end
end


#---
#write into decision function
function decide(Ny,Nb,Vd,Vr,V,decision)

    ib = (blockIdx().x-1)*blockDim().x + threadIdx().x
    iy = (blockIdx().y-1)*blockDim().y + threadIdx().y

    if (ib <= Nb && iy <= Ny)

        if (Vd[iy] < Vr[iy,ib])
            V[iy,ib] = Vr[iy,ib]
            decision[iy,ib] = 0
        else
            V[iy,ib] = Vd[iy]
            decision[iy,ib] = 1
        end
    end
    return
end

function prob_calc(Ny,Nb,prob,P,decision)
    ib = (blockIdx().x-1)*blockDim().x + threadIdx().x
    iy = (blockIdx().y-1)*blockDim().y + threadIdx().y

    if (ib <= Nb && iy <= Ny)
        #prob[iy,ib] = P[iy,:]'decision[:,ib]
        for y in Ny
            prob[iy,ib] += P[iy,y]*decision[y,ib]
        end
    end
    return
end


Price_calc(x, rstar) = (1-x) / (1+rstar)
#@benchmark Price = Price_calc.(prob, rstar)


#line 7.1 Intitializing U((1-τ)iy) to each Vd[iy] #BATCH UPDATE
function def_init_old(sumdef,τ,Y,α)
    iy = threadIdx().x
    stride = blockDim().x
    for i = iy:stride:length(sumdef)
        sumdef[i] = exp((1-τ)*Y[i])/(1-α)
    end
    return
end

#line 7.2 adding second expected part to calcualte Vd[iy]
function def_add_old(matrix, P, β, V0, Vd0, ϕ, Ny)
    y = (blockIdx().x-1)*blockDim().x + threadIdx().x
    iy = (blockIdx().y-1)*blockDim().y + threadIdx().y

    if (iy <= Ny && y <= Ny)
        matrix[iy,y] = β* P[iy,y]* (ϕ* V0[y,1] + (1-ϕ)* Vd0[y])
    end
    return
end

function vr_old(Nb,Ny,α,β,τ,Vr,V0,Y,B,Price0,P)

    ib = (blockIdx().x-1)*blockDim().x + threadIdx().x
    iy = (blockIdx().y-1)*blockDim().y + threadIdx().y

    if (ib <= Nb && iy <= Ny)

        Max = -Inf
        for b in 1:Nb
            c = Float32(CUDA.exp(Y[iy]) + B[ib] - Price0[iy,b]*B[b])
            if c > 0 #If consumption positive, calculate value of return
                sumret = 0
                for y in 1:Ny
                    sumret += V0[y,b]*P[iy,y]
                end
                Max = CUDA.max(Max, CUDA.pow(c,(1-α))/(1-α) + β * sumret)
            end
        end
        Vr[iy,ib] = Max
    end
    return
end


#line 9-14 debt price update
function Decide_old(Nb,Ny,Vd,Vr,V,decision,decision0,prob,P,Price,rstar)

    ib = (blockIdx().x-1)*blockDim().x + threadIdx().x
    iy = (blockIdx().y-1)*blockDim().y + threadIdx().y

    if (ib <= Nb && iy <= Ny)

        if (Vd[iy] < Vr[iy,ib])
            V[iy,ib] = Vr[iy,ib]
            decision[iy,ib] = 0
        else
            V[iy,ib] = Vd[iy]
            decision[iy,ib] = 1
        end

        for y in 1:Ny
            prob[iy,ib] += P[iy,y] * decision[y,ib]
        end

        Price[iy,ib] = (1-prob[iy,ib]) / (1+rstar)

    end
    return
end


function tauchen(ρ, σ, Ny, P)
    #Create equally spaced pts to fill into Z
    σ_z = sqrt((σ^2)/(1-ρ^2))
    Step = 10*σ_z/(Ny-1)
    Z = -5*σ_z:Step:5*σ_z

    #Fill in entries of 1~ny, ny*(ny-1)~ny^2
    for z in 1:Ny
        P[z,1] = cdf(Normal(), (Z[1]-ρ*Z[z] + Step/2)/σ)
        P[z,Ny] = 1 - cdf(Normal(),(Z[Ny] - ρ*Z[z] - Step/2)/σ)
    end

    #Fill in the middle part
    for z in 1:Ny
        for iz in 2:(Ny-1)
            P[z,iz] = cdf(Normal(), (Z[iz]-ρ*Z[z]+Step/2)/σ) - cdf(Normal(), (Z[iz]-ρ*Z[z]-Step/2)/σ)
        end
    end
end


tauchen (generic function with 1 method)

Change Ny and Nb below to change matrix to size Ny * Nb
To demonstrate I use 300 * 300

In [4]:
#Setting parameters
Ny = 300 #grid number of endowment
Nb = 300 #grid number of bond
maxInd = Ny * Nb #total grid points
rstar = 0.017 #r* used in price calculation
α = Float32(0.5) #α used in utility function

#lower bound and upper bound for bond initialization
lbd = -1
ubd = 0

#β,ϕ,τ used as in part 4 of original paper
β = 0.953
ϕ = 0.282
τ = 0.5

δ = 0.8 #weighting average of new and old matrixs

#ρ,σ For tauchen method
ρ = 0.9
σ = 0.025


#Initializing Bond matrix
minB = lbd
maxB = ubd
step = (maxB-minB) / (Nb-1)
B = CuArray(minB:step:maxB) #Bond

#Intitializing Endowment matrix
σ_z = sqrt((σ^2)/(1-ρ^2))
Step = 10*σ_z/(Ny-1)
Y = CuArray(-5*σ_z:Step:5*σ_z) #Endowment

Pcpu = zeros(Ny,Ny)  #Conditional probability matrix
V = CUDA.fill(1/((1-β)*(1-α)), Ny, Nb) #Value
Price = CUDA.fill(1/(1+rstar), Ny, Nb) #Debt price
Vr = CUDA.zeros(Ny, Nb) #Value of good standing
Vd = CUDA.zeros(Ny) #Value of default
decision = CUDA.ones(Ny, Nb) #Decision matrix
C = CUDA.zeros(Ny,Nb,Nb)
VR = CUDA.zeros(Ny,Nb,Nb)
sumret = CUDA.zeros(Ny,Nb,Nb)
global temp = CUDA.zeros(Ny,Ny)

#U(x) = x^(1-α) / (1-α) #Utility function, change this into a function
function U(x)
    if x >= 0
        return x^(1-α) / (1-α) #Utility function7
    end
    return 0
end
function U2(x)
    return (x>0) * (x+0im)^(1-α) / (1-α)
end
#U2.(C)

#Initialize Conditional Probability matrix
tauchen(ρ, σ, Ny, Pcpu)
P=CuArray(Pcpu)


err = 2000 #error
tol = 1e-3 #error toleration
iter = 0
maxIter = 500 #Maximum interation

V0 = CUDA.deepcopy(V)
Vd0 = CUDA.deepcopy(Vd)
Price0 = CUDA.deepcopy(Price)
prob = CUDA.zeros(Ny,Nb)
decision = CUDA.ones(Ny,Nb)
decision0 = CUDA.deepcopy(decision)
C = CUDA.zeros(Ny,Nb,Nb)
#We set up C2, sumret and sumdef in device memory
sumret = CUDA.zeros(Ny,Nb,Nb)
sumdef = CUDA.zeros(Ny)
C2 = CUDA.zeros(Ny,Nb,Nb)

threadcount = (16,16) #set up defualt thread numbers per block
blockcount = (ceil(Int,Ny/10),ceil(Int,Ny/10))

┌ Warning: `haskey(::TargetIterator, name::String)` is deprecated, use `Target(; name = name) !== nothing` instead.
│   caller = llvm_compat(::VersionNumber) at compatibility.jl:176
└ @ CUDA C:\Users\dengmingzhuo\.julia\packages\CUDA\5t6R9\deps\compatibility.jl:176
┌ Warning: `Target(triple::String)` is deprecated, use `Target(; triple = triple)` instead.
│   caller = ip:0x0
└ @ Core :-1


(30, 30)

In [7]:
#using Benchmark on old Value of Repayment calculation, matrix size 300*300
@benchmark @cuda threads=threadcount blocks=blockcount vr_old(Nb,Ny,α,β,τ,Vr,V0,Y,B,Price0,P)

BenchmarkTools.Trial: 
  memory estimate:  4.19 KiB
  allocs estimate:  106
  --------------
  minimum time:     12.200 μs (0.00% GC)
  median time:      13.000 μs (0.00% GC)
  mean time:        632.538 ms (0.00% GC)
  maximum time:     8.223 s (0.00% GC)
  --------------
  samples:          13
  evals/sample:     1

In [8]:
#Benchmark on new Value of Repayment, adding the time of three steps together
t0 = @benchmark @cuda threads=threadcount blocks=blockcount vr_sumret(Ny,Nb,V0,P,sumret)

BenchmarkTools.Trial: 
  memory estimate:  7.88 KiB
  allocs estimate:  189
  --------------
  minimum time:     72.000 μs (0.00% GC)
  median time:      75.900 μs (0.00% GC)
  mean time:        85.386 μs (1.01% GC)
  maximum time:     12.451 ms (69.02% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [8]:
t1 = @benchmark sumret .*= β

BenchmarkTools.Trial: 
  memory estimate:  1.95 KiB
  allocs estimate:  61
  --------------
  minimum time:     14.801 μs (0.00% GC)
  median time:      18.201 μs (0.00% GC)
  mean time:        702.941 ms (0.00% GC)
  maximum time:     11.247 s (0.00% GC)
  --------------
  samples:          16
  evals/sample:     1

In [6]:
t2 = @benchmark reduce(max,vr,dims=3)

BenchmarkTools.Trial: 
  memory estimate:  8.02 KiB
  allocs estimate:  198
  --------------
  minimum time:     69.200 μs (0.00% GC)
  median time:      422.500 μs (0.00% GC)
  mean time:        8.319 ms (0.00% GC)
  maximum time:     158.544 ms (0.00% GC)
  --------------
  samples:          609
  evals/sample:     1

In [ ]:
time(median(t0)) + time(median(t1)) + time(median(t2))

In [ ]:
#Using @time to get the median time
counter = 30
t=[]
    for i in 1:counter
        time = @timed @cuda threads=threadcount blocks=blockcount vr_old(Nb,Ny,α,β,τ,Vr,V0,Y,B,Price0,P)
        push!(t, time[2])
    end
median(t)